In [28]:
import mysql.connector as SQLC  

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()

def get_all_organisms ():

    cur.execute("""select Organism from Genbank""")
    organisms_DB = cur.fetchall()
    return organisms_DB

def get_seq_from_organism (organism):
    cur.execute(f""" select Sequence from Sequences
                    join Genbank on Sequences.ID_version_seq = Genbank.ID_version_seq
                    where Organism = '{organism}'
    """)
    sequence = cur.fetchall()
    return sequence[0][0]

def get_all_seqs():
    cur.execute(f""" select Sequence from Sequences
    """)
    sequence = cur.fetchall()
    return sequence[0]





In [37]:
get_all_organisms()
org = get_seq_from_organism('Sulfolobus spindle-shaped virus 1')
str_bd = ""
for c in get_all_seqs():
    str_bd += c
str_bd
teste = get_all_seqs()[0]

In [22]:
class Blast ():
    def __init__(self, query, seq, w = 3):
        self.seq = seq
        self.query = query
        self.w = w


    def query_map(query, w):
            tam = len(query)
            res = {}
            for chave, offset in [(query[p : p + w], p) for p in range(0, tam - w +1)]:
                if chave not in res:
                    res[chave] = []
                res[chave].append(offset)
            return(res)


    def get_all_offsets(s1, s2):  # s1 --> Padrão    s2 --> seq_analisar
        w = len(s1)
        return[p for p in range(0, len(s2) - w + 1) if s2[p : p + w] == s1]


    def hits(qm, seq):      # qm --> query_map(query, w)   
        return[(o_query, o_seq) for chave, offsets in qm.items() 
        for o_query in offsets 
        for o_seq in Blast.get_all_offsets(chave, seq)]


    def extend_hit_any_direction(query, seq, o1, o2, dir):
        matches = 0
        count = 0
        while o1 >= 0 and o2 >= 0 and o1 < len(query) and o2 < len(seq):
            matches += 1 if query[o1] == seq[o2] else 0
            count += 1
            if 2 * matches < count:
                return o1, o2, matches, count
            o1 += dir
            o2 += dir
        return o1 - dir, o2 - dir, matches, count


    def extend_hit(query, seq, hit, w):

        o1, o2 = hit
        left  = Blast.extend_hit_any_direction(query, seq, o1 - 1, o2 - 1, -1)
        right = Blast.extend_hit_any_direction(query, seq, o1 + w, o2 + w, +1)

        O1, O2, ML, SL = left
        _,   _, MR, SR = right
        return O1, O2, w + SL + SR, ML + w + MR
    

    def best_hit (self):
        qm = Blast.query_map(self.query,self.w)
        lista_hits = Blast.hits(qm, self.seq)
        all_scores =[Blast.extend_hit(self.query, self.seq, c, self.w) for c in lista_hits]
        all_scores.sort(key=lambda c: c[2] and c[3], reverse=True)
        if all_scores == []:
            return print("Not Found")
        else:
            return all_scores [0]

In [41]:
query = "AATATAT"
seq = "AATATGTTATATAATAATATTT"
w = 3
a = Blast(query, seq, w)
a.best_hit()

# b = Blast(org,teste,3)
# b.best_hit()


(0, 0, 7, 6)